# Casos confirmados - gráficos
> Gráficos de acumulado de infectados confirmados.

- toc: false 
- badges: false
- comments: false

Última actualización 07/03/2020 - 00:22

Los datos serán actualizados cada noche, refresca el navegador si ves gráficas desactualizadas.

## Total

In [1]:
#collapse

import requests
import pandas as pd
from io import BytesIO
import numpy as np
import altair as alt

# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1FwOXbkWeuS1LHrKxhSfepGtsgQ6iIEQZmWig_JKo8z0&output=csv')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data), parse_dates=['Fecha'], dayfirst=True)

# Create a selection that chooses the nearest point & selects based on x-value
source = df
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Total confirmados'], empty='none')

# The basic line
line = alt.Chart(source).mark_line(interpolate='basis').encode(
    x='monthdate(Fecha):O',
    y='Total confirmados:Q',
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='monthdate(Fecha):O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Total confirmados:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='monthdate(Fecha):O',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=400
).configure(
    padding=10
)

alt.LayerChart(...)

## Por comunidad autónoma

Haz click en la leyenda para activar una única línea.

In [2]:
#collapse

import requests
import pandas as pd
from io import BytesIO
import numpy as np
import altair as alt

# data download
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1FwOXbkWeuS1LHrKxhSfepGtsgQ6iIEQZmWig_JKo8z0&output=csv')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data), parse_dates=['Fecha'], dayfirst=True)

# plot
cols = list(df.columns)
cols.remove('Total confirmados')
cols.remove('Fecha')
df = df.set_index('Fecha')[cols]
source = df.reset_index().melt('Fecha', var_name='category', value_name='Confirmados')

selection = alt.selection_multi(fields=['category'], bind='legend')

line = alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('monthdate(Fecha):O', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
)

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Fecha'], empty='none')

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='monthdate(Fecha):O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Confirmados:Q', alt.value(' '))
)

line = line.add_selection(
    selection
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='monthdate(Fecha):O',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=400
).configure(
    padding=10
)

alt.LayerChart(...)

## Comparación con otros países

Haz click en la leyenda para activar una única línea.

Datos tomados de fuentes externas. Ver sección "Sobre esta herramienta" para más información.

In [3]:
# collapse

# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')

# plot

# color scheme
def my_theme():
    return {
        'config': {
            'range': {'category': {'scheme': 'category10'}}
        }
    }

# Register and enable.
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')

keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore', 'Germany']
df = df[keep]
# the last day is dicarded until the day is (almost) over
plus_all = 55
df = df[0:plus_all]

source = df.reset_index().melt('Fecha', var_name='category', value_name='Confirmados')
selection = alt.selection_multi(fields=['category'], bind='legend')

line = alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('monthdate(Fecha):O', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
)

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Fecha'], empty='none')

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='monthdate(Fecha):O',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Confirmados:Q', alt.value(' '))
)

line = line.add_selection(
    selection
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='monthdate(Fecha):O',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=400
).configure(
    padding=10
)

alt.LayerChart(...)

## Comparación con otros países alineando las curvas temporalmente

### Hasta la actualidad e incluyendo los 20 primeros días de China

Haz click en la leyenda para activar una única línea.

In [26]:
# collapse
# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')
keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore', 'Germany', 'China']
df = df[keep]

# align data
aligned_data = {}
daily_delta = 0
plus_days = 23 + daily_delta
plus_spain = 11 + daily_delta
plus_iran = 19 + daily_delta
plus_italy = 19 + daily_delta
plus_france = 13 + daily_delta
plus_germany = 13 + daily_delta
plus_southkorea = 22 + daily_delta
plus_china = 21

aligned_data['Spain'] = np.array(df['Spain'][44:44+plus_spain])
aligned_data['Italy'] = np.array(df['Italy'][36:36+plus_italy])
aligned_data['South Korea'] = np.array(df['South Korea'][33:33+plus_southkorea])
aligned_data['Iran'] = np.array(df['Iran'][39:39+plus_iran])
aligned_data['Japan'] = np.array(df['Japan'][32:32+plus_days])
aligned_data['France'] = np.array(df['France'][42:42+plus_france])
aligned_data['Singapore'] = np.array(df['Singapore'][32:32+plus_days])
aligned_data['Germany'] = np.array(df['Germany'][42:42+plus_germany])
aligned_data['China'] = np.array(df['China'][0:0+plus_china])
df_aligned = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in aligned_data.items() ]))
df = df_aligned

# plot
# color scheme
def my_theme():
    return {
        'config': {
            'range': {'category': {'scheme': 'category10'}}
        }
    }

# Register and enable.
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')

source = df.reset_index().melt('index', var_name='category', value_name='Confirmados')
source.index = list(source.index)
source.index = source.index.astype(int)
selection = alt.selection_multi(fields=['category'], bind='legend')

line = alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('index:Q', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
)

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['index'], empty='none')

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='index:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Confirmados:Q', alt.value(' '))
)

line = line.add_selection(
    selection
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='index:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=400
)

alt.LayerChart(...)

### Alineación alternativa de España, Italia, Francia, Alemania

In [35]:
# collapse
# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')
keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore', 'Germany', 'China']
df = df[keep]

# align data
aligned_data = {}
daily_delta = 0
plus_days = 23 + daily_delta
plus_spain = 11 + daily_delta
plus_iran = 19 + daily_delta
plus_italy = 19 + daily_delta
plus_france = 12 + daily_delta
plus_germany = 11 + daily_delta
plus_southkorea = 22 + daily_delta
plus_china = 21

aligned_data['Spain'] = np.array(df['Spain'][44:44+plus_spain])
aligned_data['Italy'] = np.array(df['Italy'][36:36+plus_italy])
aligned_data['France'] = np.array(df['France'][43:43+plus_france])
aligned_data['Germany'] = np.array(df['Germany'][43:43+plus_germany])
df_aligned = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in aligned_data.items() ]))
df = df_aligned

# plot
# color scheme
def my_theme():
    return {
        'config': {
            'range': {'category': {'scheme': 'category10'}}
        }
    }

# Register and enable.
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')

source = df.reset_index().melt('index', var_name='category', value_name='Confirmados')
source.index = list(source.index)
source.index = source.index.astype(int)
selection = alt.selection_multi(fields=['category'], bind='legend')

line = alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('index:Q', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
)

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['index'], empty='none')

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='index:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Confirmados:Q', alt.value(' '))
)

line = line.add_selection(
    selection
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='index:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=400
)

alt.LayerChart(...)

### Hasta la actualidad

In [14]:
# collapse
# get data
response = requests.get('https://docs.google.com/spreadsheet/ccc?key=1Z7VQ5xlf3BaTx_LBBblsW4hLoGYWnZyog3jqsS9Dbgc&output=csv&gid=0')
assert response.status_code == 200, 'Wrong status code'
data = response.content
df = pd.read_csv(BytesIO(data))
df = df[df.columns[2:]][1:65].T.reset_index(drop=True)
col_names = df.iloc[0]
col_names[1] = 'Fecha'
df.columns = col_names
df = df.drop(df.index[0])
df['Fecha'] = pd.to_datetime(df['Fecha'], utc=True)
df = df.set_index('Fecha')
df = df.applymap(lambda a: str(a).replace(',',''))
df = df.apply(pd.to_numeric, errors='coerce')
keep = ['Spain', 'Italy', 'South Korea', 'Iran', 'Japan', 'France', 'Singapore', 'Germany', 'China']
df = df[keep]

# align data
aligned_data = {}
daily_delta = 0
plus_days = 23 + daily_delta
plus_spain = 11 + daily_delta
plus_iran = 6 + daily_delta
plus_italy = 19 + daily_delta
plus_france = 13 + daily_delta
plus_germany = 13 + daily_delta
plus_southkorea = 22 + daily_delta
plus_china = 49 + daily_delta

aligned_data['Spain'] = np.array(df['Spain'][44:44+plus_spain])
aligned_data['Italy'] = np.array(df['Italy'][36:36+plus_italy])
aligned_data['South Korea'] = np.array(df['South Korea'][33:33+plus_southkorea])
aligned_data['Iran'] = np.array(df['Iran'][39:39+plus_iran])
aligned_data['Japan'] = np.array(df['Japan'][32:32+plus_days])
aligned_data['France'] = np.array(df['France'][42:42+plus_france])
aligned_data['Singapore'] = np.array(df['Singapore'][32:32+plus_days])
aligned_data['Germany'] = np.array(df['Germany'][42:42+plus_germany])
aligned_data['China'] = np.array(df['China'][0:0+plus_china])
df_aligned = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in aligned_data.items() ]))
df = df_aligned

# plot
# color scheme
def my_theme():
    return {
        'config': {
            'range': {'category': {'scheme': 'category10'}}
        }
    }

# Register and enable.
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')

source = df.reset_index().melt('index', var_name='category', value_name='Confirmados')
source.index = list(source.index)
source.index = source.index.astype(int)
selection = alt.selection_multi(fields=['category'], bind='legend')

line = alt.Chart(source).mark_line(interpolate='basis').encode(
    alt.X('index:Q', axis=alt.Axis(tickRound=True), title='Número de días'),
    alt.Y('Confirmados:Q'),
    alt.Color('category:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
)

nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['index'], empty='none')

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='index:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'Confirmados:Q', alt.value(' '))
)

line = line.add_selection(
    selection
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='index:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=800, height=400
)

alt.LayerChart(...)